In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.utils import class_weight

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers, regularizers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pickle
import pathlib
import hashlib
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dotenv
from dotenv import dotenv_values


import visualkeras
from PIL import ImageFont
from collections import defaultdict

# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_unet import *
from utils.utils_resnet import *
from utils.utils_plot import *

Num GPUs Available:  1


In [2]:
config = dotenv_values(".env")

# Paths
PATH_ERA5 = config['PATH_ERA5']
PATH_EOBS = config['PATH_EOBS']


In [3]:
out_cropping=None

In [4]:
l_models = os.listdir('tmp')

In [5]:
#l_models

In [6]:
# this is extremely slow!
#reconstructed_model = tf.keras.models.load_model('tmp/' + l_models[0])

In [7]:
class DeepFactory(tf.keras.Model):
    """
    Model factory.
    """

    def __init__(self, arch, input_size, output_size, for_extremes=False, latent_dim=128, dropout_rate=0.2, 
                 use_batch_norm=True, inner_activation='relu', output_scaling=1, output_crop=None):
        super(DeepFactory, self).__init__()
        self.arch = arch
        self.input_size = list(input_size)
        self.output_size = list(output_size)
        self.for_extremes = for_extremes
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        self.use_batch_norm = use_batch_norm
        self.inner_activation = inner_activation
        self.output_scaling = output_scaling
        self.output_crop = output_crop
        
        self.last_activation = 'relu'
        if for_extremes:
            self.last_activation = 'sigmoid'

        if arch == 'Davenport-2021':
            self.build_Davenport_2021()
        elif arch == 'CNN-2L':
            self.build_CNN_2L()
        elif arch == 'Unet':
            self.build_Unet()
        elif arch == 'Pan-2019':
            self.build_Pan_2019()
        elif arch =='Conv-LTSM':
            self.build_convLTSM()
        else:
            raise ValueError('The architecture was not correctly defined')
        
        
    def build_Davenport_2021(self):
        """
        Based on: Davenport, F. V., & Diffenbaugh, N. S. (2021). Using Machine Learning to 
        Analyze Physical Causes of Climate Change: A Case Study of U.S. Midwest Extreme Precipitation. 
        Geophysical Research Letters, 48(15). https://doi.org/10.1029/2021GL093787
        """
        
        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        x = layers.Conv2D(16, 3, padding='same', activity_regularizer=regularizers.l2(0.01))(inputs)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.SpatialDropout2D(self.dropout_rate)(x) # In original: simple Dropout
        x = layers.Conv2D(16, 3, padding='same', activity_regularizer=regularizers.l2(0.01))(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.SpatialDropout2D(self.dropout_rate)(x) # In original: simple Dropout
        x = layers.Flatten()(x)                
        x = layers.Dense(self.latent_dim, activity_regularizer=regularizers.l2(0.001))(x) # In original: 16
        x = layers.Activation('relu')(x)

        next_shape = self.get_shape_for(stride_factor=4)

        # Upsampling. In original: no decoder
        x = self.dense_block(x, np.prod(next_shape))
        x = layers.Reshape(target_shape=next_shape)(x)
        x = self.deconv_block(x, 16, 3, stride=2)
        x = self.deconv_block(x, 16, 3, stride=2)
        x = self.conv_block(x, 1, 3, activation=self.last_activation)
        outputs = self.final_cropping_block(x)
 
        self.model = keras.Model(inputs, outputs, name="Davenport-2021")
        
        
    def build_Pan_2019(self):
        """
        Based on: Pan, B., Hsu, K., AghaKouchak, A., & Sorooshian, S. (2019). 
        Improving Precipitation Estimation Using Convolutional Neural Network. 
        Water Resources Research, 55(3), 2301–2321. https://doi.org/10.1029/2018WR024090
        """
        # In original: padding='valid'
        
        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        x = layers.Conv2D(15, 4, padding='same')(inputs)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(20, 4, padding='same')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.Conv2D(20, 4, padding='same')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.Flatten()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(self.latent_dim, activation='relu')(x) # In original: 60

        next_shape = self.get_shape_for(stride_factor=4)

        # Upsampling. In original: no decoder
        x = self.dense_block(x, np.prod(next_shape))
        x = layers.Reshape(target_shape=next_shape)(x)
        x = self.deconv_block(x, 20, 4, stride=2)
        x = self.deconv_block(x, 20, 4, stride=2)
        x = self.conv_block(x, 15, 4)
        x = self.conv_block(x, 1, 3, activation=self.last_activation)
        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="Pan-2019")


    def build_CNN_2L(self):

        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        x = self.conv_block(inputs, 32, 3, stride=2, with_batchnorm=True, with_dropout=True)
        x = self.conv_block(x, 64, 3, stride=2, with_batchnorm=True, with_dropout=True)
        x = layers.Flatten()(x)
        x = self.dense_block(x, self.latent_dim, activation='sigmoid')

        next_shape = self.get_shape_for(stride_factor=4)

        # Upsampling
        x = self.dense_block(x, np.prod(next_shape))
        x = layers.Reshape(target_shape=next_shape)(x)
        x = self.deconv_block(x, 64, 3, stride=2)
        x = self.deconv_block(x, 32, 3, stride=2)
        x = self.deconv_block(x, 1, 3, activation=self.last_activation)
        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="CNN-v1")
        

    def build_convLTSM(self):
        
        expanded_shape = self.input_size.copy()
        expanded_shape.insert(0, 1)
        
        inputs = layers.Input(shape=self.input_size)
        
        x = layers.Reshape(target_shape=expanded_shape)(inputs)
        
        x = layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), return_sequences=True, padding = 'same',
                              go_backwards=True, activation='tanh', data_format = 'channels_last',
                              dropout=0.4, recurrent_dropout=0.2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), return_sequences=True, padding = 'same',
                              go_backwards=True, activation='tanh', data_format = 'channels_last',
                              dropout=0.4, recurrent_dropout=0.2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.ConvLSTM2D(filters=8, kernel_size=(3, 3), return_sequences=False, padding = 'same',
                              go_backwards=True, activation='tanh', data_format = 'channels_last',
                              dropout=0.3, recurrent_dropout=0.2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.Conv2D(filters=16, kernel_size=(1, 1), activation='relu', data_format='channels_last')(x)

        x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        #model.add(Flatten())
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.25)(x)

        x = layers.Conv2DTranspose(filters=1, kernel_size=(1, 1), strides=(2, 2),  activation='sigmoid',
                                   padding='same', data_format='channels_last')(x)

        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="Conv-LTSM")
        
        
    def build_Unet(self):
        """
        Based on: U-Net: https://github.com/nikhilroxtomar/Unet-for-Person-Segmentation/blob/main/model.py
        """
        
        filters_nb = 64
        
        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        
        # Pad if necessary
        x = self.padding_block(inputs, factor=16)
        
        s1, p1 = self.unet_encoder_block(x, filters_nb)
        s2, p2 = self.unet_encoder_block(p1, filters_nb * 2)
        s3, p3 = self.unet_encoder_block(p2, filters_nb * 4)
        s4, p4 = self.unet_encoder_block(p3, filters_nb * 8)
        
        x = self.conv_block(p4, filters_nb * 16, 3, initializer='he_normal', with_batchnorm=True, with_dropout=True)
        b1 = self.conv_block(x, filters_nb * 16, 3, initializer='he_normal', with_batchnorm=True)

        # Upsampling
        d1 = self.unet_decoder_block(b1, s4, filters_nb * 8)
        d2 = self.unet_decoder_block(d1, s3, filters_nb * 4)
        d3 = self.unet_decoder_block(d2, s2, filters_nb * 2)
        d4 = self.unet_decoder_block(d3, s1, filters_nb, is_last=True)
        
        # Additional upsampling for downscaling
        x = self.handle_output_scaling(d4)

        x = self.conv_block(x, 1, 1, activation=self.last_activation)
        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="U-Net-v1")
        
        
    def unet_encoder_block(self, input, filters, kernel_size=3):
        x = self.conv_block(input, filters, kernel_size, initializer='he_normal', with_batchnorm=True, with_dropout=True)
        x = self.conv_block(x, filters, kernel_size, initializer='he_normal', with_batchnorm=True)
        p = layers.MaxPooling2D((2, 2))(x)
        
        return x, p

    
    def unet_decoder_block(self, input, skip_features, filters, conv_kernel_size=3, deconv_kernel_size=2, is_last=False):
        x = self.deconv_block(input, filters, deconv_kernel_size, stride=2)
        x = layers.Concatenate()([x, skip_features])
        x = self.conv_block(x, filters, conv_kernel_size, initializer='he_normal', with_batchnorm=True, with_dropout=True)
        x = self.conv_block(x, filters, conv_kernel_size, initializer='he_normal', with_batchnorm=(not is_last))

        return x
        
        
    def conv_block(self, input, filters, kernel_size=3, stride=1, padding='same', initializer='default', activation='default', 
                   with_batchnorm=False, with_pooling=False, with_dropout=False, with_late_activation=False):
        if activation == 'default':
            activation = self.inner_activation
            
        conv_activation = activation
        if with_late_activation:
            conv_activation = None
            
        if initializer == 'default':
            x = layers.Conv2D(filters, kernel_size, strides=(stride, stride), padding=padding, activation=conv_activation)(input)
        else:
            x = layers.Conv2D(filters, kernel_size, strides=(stride, stride), padding=padding, activation=conv_activation, kernel_initializer=initializer)(input)
            
        if with_batchnorm:
            x = layers.BatchNormalization()(x)
        if with_late_activation:
            x = layers.Activation(activation)(x)
        if with_pooling:
            x = layers.MaxPooling2D(pool_size=2)(x)
        if with_dropout:
            x = layers.SpatialDropout2D(self.dropout_rate)(x)
        
        return x
    
    
    def deconv_block(self, input, filters, kernel_size=3, stride=1, padding='same', initializer='default', activation='default', 
                     with_batchnorm=False, with_dropout=False):
        if activation == 'default':
            activation = self.inner_activation
        
        if initializer == 'default':
            x = layers.Conv2DTranspose(filters, kernel_size, strides=stride, padding=padding, activation=activation)(input)
        else:
            x = layers.Conv2DTranspose(filters, kernel_size, strides=stride, padding=padding, activation=activation, kernel_initializer=initializer)(input)
            
        if with_batchnorm:
            x = layers.BatchNormalization()(x)
        if with_dropout:
            x = layers.SpatialDropout2D(self.dropout_rate)(x)
        
        return x
    
    
    def dense_block(self, input, units, activation='default', with_dropout=False):
        if activation == 'default':
            activation=self.inner_activation
            
        x = layers.Dense(units, activation=activation)(input)
        if with_dropout:
            x = layers.Dropout(self.dropout_rate)(x)
            
        return x

    
    def handle_output_scaling(self, input, with_batchnorm=False):
        if self.output_scaling > 1:
            if self.output_scaling == 2:
                x = self.deconv_block(input, 64, 3, stride=2, with_batchnorm=with_batchnorm)
            elif self.output_scaling == 3:
                x = self.deconv_block(input, 64, 3, stride=3, with_batchnorm=with_batchnorm)
            elif self.output_scaling == 4:
                x = self.deconv_block(input, 64, 3, stride=2, with_batchnorm=with_batchnorm)
                x = self.deconv_block(x, 64, 3, stride=2, with_batchnorm=with_batchnorm)
            elif self.output_scaling == 5:
                x = self.deconv_block(input, 64, 3, stride=3, with_batchnorm=with_batchnorm)
                x = self.deconv_block(x, 64, 3, stride=2, with_batchnorm=with_batchnorm)
            else:
                raise NotImplementedError('Level of downscaling not implemented')
        else:
            x = input
        
        if self.output_crop:
            raise NotImplementedError('Manual cropping not yet implemented')
            
        return x
            
        
    def padding_block(self, x, factor):
        h, w = x.get_shape().as_list()[1:3]
        dh = 0
        dw = 0
        if h % factor > 0:
            dh = factor - h % factor
        if w % factor > 0:
            dw = factor - w % factor
        if dh > 0 or dw > 0:
            top_pad = dh//2
            bottom_pad = dh//2 + dh%2
            left_pad = dw//2
            right_pad = dw//2 + dw%2
            x = layers.ZeroPadding2D(padding=((top_pad, bottom_pad), (left_pad, right_pad)))(x)
        
        return x
        
        
    def final_cropping_block(self, x):
        # Compute difference between reconstructed width and hight and the desired output size.
        h, w = x.get_shape().as_list()[1:3]
        h_tgt, w_tgt = self.output_size[:2]
        dh = h - h_tgt
        dw = w - w_tgt

        if dh < 0 or dw < 0:
            raise ValueError(f'Negative values in output cropping dh={dh} and dw={dw}')

        # Add to decoder cropping layer and final reshaping
        x = layers.Cropping2D(cropping=((dh//2, dh-dh//2), (dw//2, dw-dw//2)))(x)
        #x = layers.Reshape(target_shape=self.output_size,)(x)
        
        return x
        

    def get_shape_for(self, stride_factor):
        next_shape = self.output_size.copy()
        next_shape[0] = int(np.ceil(next_shape[0]/stride_factor))
        next_shape[1] = int(np.ceil(next_shape[1]/stride_factor))

        return next_shape

        
    def call(self, x):
        return self.model(x)
    


In [8]:
models = {
          'Dav-orig': {'model': 'Davenport-2021', 'run': True,
                       'opt_model': {'latent_dim': 16},
                       'opt_optimizer': {'lr_method': 'Constant'}}, # original
          'Dav-64': {'model': 'Davenport-2021', 'run': True,
                     'opt_model': {'latent_dim': 64},
                     'opt_optimizer': {'lr_method': 'Constant'}},
          'Pan-orig': {'model': 'Pan-2019', 'run': True,
                       'opt_model': {'latent_dim': 60},
                       'opt_optimizer': {'lr_method': 'Constant', 'lr': 1e-4}},
          'CNN-2l': {'model': 'CNN-2L', 'run': True,
                     'opt_model': {'latent_dim': 64},
                     'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': 1, 'output_crop': None},
                   'opt_optimizer': {'lr_method': 'CosineDecay'}}
         
         }

In [9]:
# Just for plotting
i_shape = [46,56,10]
o_shape = [46,56,1]
opt_model = {'latent_dim': 128,
             'dropout_rate': 0.2}
opt_optimizer = 'adam'

In [10]:
l_mod=[]
color_map = defaultdict(dict)
color_map[Conv2D]['fill'] = 'orange'
color_map[Dropout]['fill'] = 'silver'
color_map[MaxPooling2D]['fill'] = 'red'
color_map[Dense]['fill'] = 'blue'
color_map[Flatten]['fill'] = 'purple'
color_map[BatchNormalization]['fill'] = 'lightgreen'
for m_id in models:
    # Extract model name and options
    model = models[m_id]['model']
    print(model)
    opt_model_i = models[m_id]['opt_model']
    opt_optimizer_i = models[m_id]['opt_optimizer']
    opt_model_new = opt_model.copy()
    opt_model_new.update(opt_model_i)
    opt_optimizer_new = opt_optimizer
    
    m = DeepFactory(model, i_shape, o_shape, for_extremes=False, **opt_model_new)
    # Plot
    #visualkeras.layered_view(m.model, legend=True,color_map=color_map,
    #                     type_ignore=[ZeroPadding2D, Reshape, Dropout, BatchNormalization, Activation],to_file='figures/arch'+model+'.png')
    l_mod.append(m)

Davenport-2021


2022-06-23 09:41:26.901355: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-23 09:41:27.698058: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:07:00.0, compute capability: 6.1


Davenport-2021
Pan-2019
CNN-2L
Unet


In [11]:
l_mod[0].model.summary()

Model: "Davenport-2021"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 46, 56, 10)]      0         
                                                                 
 conv2d (Conv2D)             (None, 46, 56, 16)        1456      
                                                                 
 activation (Activation)     (None, 46, 56, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 28, 16)       0         
 )                                                               
                                                                 
 spatial_dropout2d (SpatialD  (None, 23, 28, 16)       0         
 ropout2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 28, 16)     